In [3]:
import pandas as pd

In [4]:
file_txt = open('wr112570.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)

file_csv.close()
file_txt.close()

'''Создаем файл в формате csv'''

'Создаем файл в формате csv'

In [5]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)

'''Создаем датафрейм'''

'Создаем датафрейм'

In [6]:
# df_filtered = df[(df['month'] > 5) & (df['month'] < 9)]

# '''Оставляем только нужные месяцы летнего сезона: июнь, июль, август'''

In [7]:
df_cleaned = df.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_cleaned.dropna(inplace=True)
df_cleaned['station'] = df_cleaned['station'].astype(int)

df_cleaned.to_csv('cleaned_dataset_year.csv',  index=False)

'''Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '''

'Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '

In [8]:
df_first = pd.read_csv('cleaned_dataset_year.csv')
df_second = pd.read_csv('stations.csv')
df_new = df_first.merge(df_second, how='left', on='station')

'''Добавляем данные о регионе метеостанаций'''
df_new.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
df_new['number of region'] = df_new['number of region'].astype(int)

In [9]:
"""Считаем климатическую норму для периода 1960-2009"""

norma = round(df_new[(df_new['year'] > 1960) & (df_new['year'] < 1991)].groupby(['day','month','station'])['max_temperature'].mean(), 1)
norma
df_norma = pd.DataFrame(norma, columns=['max_temperature']).reset_index()
df_norma.rename(columns={'max_temperature':'climatic_norm'}, inplace=True)
df_climatic_norm = df_new.merge(df_norma, how='outer', on=['day', 'month','station'])

In [10]:
'''Смотрим разницу среднесуточных температур и климатической нормы в период с 2010 по 2019 гг.'''

df_result = df_climatic_norm[(df_climatic_norm['year'] > 2009) & (df_climatic_norm['year'] < 2020)]
df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)
df_result.to_csv('anomalies_season.csv',  index=False)



/var/folders/pf/sj5w0_zs1fs2qhqbvvcnn2500000gn/T/ipykernel_52929/268871915.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)


In [12]:
'''Считаем среднее температурное отклонение за год для всех регионов России в период с 2010 по 2019 гг.'''
df_year = pd.DataFrame(df_result.groupby(['region','station','year']).sum()['difference']).reset_index()



def average(row):
    if row.year == 2012 or row.year == 2016 or row.year == 2020:
        year_average = row.difference / 366
    else:
        year_average = row.difference / 365
    return year_average

df_year['year_average'] = df_year.apply(lambda row: average(row), axis=1)

df_region_average_station = pd.DataFrame(df_year.groupby(['region', 'year']).count()['station'])
df_region_average_year = pd.DataFrame(df_year.groupby(['region', 'year']).sum()['year_average'])
df_united = df_region_average_station.merge(df_region_average_year, how='outer', on=['region', 'year'])
df_united['mean_year_temperature_deviation'] = round(df_united['year_average'] / df_united['station'], 2)
df_result_mean = df_united.drop(columns=['station', 'year_average'], axis=1).reset_index()
df_result_mean.to_excel('mean_year_temperature_deviation.xlsx',  index=True)
df_result_mean.to_csv('mean_year_temperature_deviation.csv',  index=True)
